# Message Authentication Codes

# Introduction

Message Authentication Codes are used to address two main information security objectives:

- **Authentication**. 
    - Is the message received by Bob really from Alice?
- **Data Intgerity** 
    - Has the received message been altered during transmission?

To verify these, Alice sends an appendix $MAC$ to along with the message $m$.